In [1]:
'''
import necessary packages and functions
'''

import os
import re
import io
import csv
import cv2
import random
import sys
import glob
import re
import imageio
import itertools

import numpy as np
import matplotlib.pyplot as plt

from scipy import ndarray, ndimage
from skimage import io, color, transform, util, morphology, measure, filters
from skimage.color import rgb2gray
from skimage.io import imsave, imshow, imread
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.io import imshow, imsave
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.segmentation import clear_border
from skimage.morphology import ball, disk, remove_small_objects
from skimage.morphology import erosion, dilation, closing, reconstruction
from skimage.morphology import binary_erosion, binary_dilation, binary_closing, binary_opening
from skimage.measure import label, regionprops, perimeter
from skimage.filters import roberts, sobel, threshold_otsu


In [2]:
'''
define import_images function to pull from directory, convert to grayscale, append to list
'''
def import_images(dir_name, images, image_names):
    for root, dirnames, filenames in os.walk(dir_name):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
                filepath = os.path.join(root, filename)
                image_names.append(filename)
                
                ## read images and add to list
                image = io.imread(fname=filepath)
                image_gray = rgb2gray(image)
                images.append(image_gray)
                
'''
define move_images function to move files between directories
'''            
def move_images(img_list, src_dir, dest_dir):
    for img in img_list:
        src = os.path.join(src_dir, img)
        dest = os.path.join(dest_dir, img)
        os.rename(src, dest)
        
'''
define lung segmentation function
'''

def segment_lungs(im):
    """
    Input: 2D lung CT image in the form of an array
    Output: binary mask for lung CT
    """
    
    ##### BINARIZE IMAGE
    # first anything <0 (as some images are negative)
    im_b = np.where(im<0, 0, im)
    #then based on otsu thresholding
    thresh = threshold_otsu(im_b)
    binary = im_b > thresh

    #invert the image to make the lungs the ROIs
    binary = np.invert(binary)

    ##### GENERATE BORDER OF THE BINARIES
    cleared = clear_border(binary)
    
    ##### LABEL DISTINCT BODIES IN IMAGE
    label_image = label(cleared)

    ##### KEEP TWO LARGEST AREAS
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0
    binary = label_image > 0
    
    ##### EROSION TO GET RID OF ARTIFACTS
#    selem = disk(4)
#    binary = binary_erosion(binary, selem)

    ##### DILATION TO BRING BACK LUNG INFORMATION
    selem = disk(10)
    binary = binary_dilation(binary, selem)

    ##### FILL SMALL HOLES
    edges = sobel(binary)
    binary = ndimage.binary_fill_holes(edges)

    return binary

'''
define function to create lung masks for all images in collection
'''

def create_masks(images, image_masks):
    for i in range(len(images)):
        img = images[i]
        img_mask = segment_lungs(img)
        image_masks.append(img_mask)

'''
define function to save all lung masks to file
'''
def save_masks(dir_out, image_masks, image_names):
    # loop through image_masks, convert from bool to save to file
    for i in range(len(image_masks)):
        img_mask = image_masks[i]
        img_name = image_names[i]

        # define destination filename
        filename='{}{}'.format(dir_out, img_name)

        # convert bool image masks to uint8 to allow writing to file
        img_mask = img_mask.astype(np.uint8)  #convert to an unsigned byte
        img_mask*=255

        # write to file
        cv2.imwrite(filename, img_mask)

'''
define split_dataset function for training, testing, and validation split
'''        
def split_dataset(imgs, src_dir, train_dir, test_dir, validation_dir, rseed = 420):
    imgs.sort
    random.seed(rseed)
    random.shuffle(imgs)
    split_1 = int(0.8 * len(imgs))
    split_2 = int(0.9 * len(imgs))
    train = imgs[:split_1]
    validation = imgs[split_1:split_2]
    test = imgs[split_2:]

    move_images(train, src_dir, train_dir)
    move_images(test, src_dir, test_dir)
    move_images(validation, src_dir, validation_dir)

'''
define image augmentation functions to amplify dataset
'''
def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

def vertical_flip(image_array: ndarray):
    # vertical flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[::-1, :]

def augment_images(imgs, img_names):
    for i in range(len(imgs)):
        img_name = img_names[i]

        himg = horizontal_flip(imgs[i])
        hfilename='{}{}{}'.format(aug_dir, 'himg', img_name)
        cv2.imwrite(hfilename, himg)

        vimg = vertical_flip(imgs[i])
        vfilename='{}{}{}'.format(aug_dir, 'vimg', img_name)
        cv2.imwrite(vfilename, vimg)

        hvimg = horizontal_flip(vimg)
        hvfilename='{}{}{}'.format(aug_dir, 'hvimg', img_name)
        cv2.imwrite(hvfilename, hvimg)
        
'''
define function to display multiple images in a single plot
'''

def show_images(images, cols = 4, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title, fontsize = 100)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()
    

        

In [ ]:
'''
import images, create masks, add to lists
'''
# define source directory for image dataset
img_dir = '../../dataset/images'

# establish empty lists for images and image names
images = []
image_names = []
    
# import images and populate lists (image names and arrays)
import_images(img_dir, images, image_names)



In [ ]:
'''
create lung masks on all images and add to list
'''
# establish empty list for lung masks
image_masks = []

# create lung masks from images list
create_masks(images, image_masks)



In [ ]:
'''
save lung masks to file
'''
# define destination directory for image masks
dir_out = '../../dataset/masks/'

# save lung masks to disk
save_masks(dir_out, image_masks, image_names)


In [5]:
'''
move images to 0/1 folders based on labels
'''

SOURCE_ROOT = '../../data_culled/images/'
DEST_ROOT = '../../data_culled/'

with open('../../data_culled/image-labels.csv') as infile:
    next(infile)  # Skip the header row
    reader = csv.reader(infile)
    seen = set()
    for External_ID, Label in reader:
        src = os.path.join(SOURCE_ROOT, External_ID)
        dest = os.path.join(DEST_ROOT, Label, External_ID)
        try:
            os.rename(src, dest)
        except WindowsError as e:
            print (e)
            

In [6]:
'''
move images to appropriate folder for training, testing, and validation
'''
yes_dir = '../../data_culled/1/'
yes_imgs = []
yes_names = []
import_images(yes_dir, yes_imgs, yes_names)

yes_train = '../../data_culled/train/1/'
yes_test = '../../data_culled/test/1/'
yes_validation = '../../data_culled/validation/1/'
split_dataset(yes_names, yes_dir, yes_train, yes_test, yes_validation)

no_dir = '../../data_culled/0/'
no_imgs = []
no_names = []
import_images(no_dir, no_imgs, no_names)

no_train = '../../data_culled/train/0/'
no_test = '../../data_culled/test/0/'
no_validation = '../../data_culled/validation/0/'
split_dataset(no_names, no_dir, no_train, no_test, no_validation)

    

In [114]:
'''
amplify training dataset via image augmentation
apply rotation and both horizontal and vertical flips to all images in train folders
'''
img_dir = '../../data_culled/train/1'
aug_dir = '../../data_culled/augmented/1/'
imgs = []
img_names = []

import_images(img_dir, imgs, img_names)
augment_images(imgs, img_names)

img_dir = '../../data_culled/train/0'
aug_dir = '../../data_culled/augmented/0/'
imgs = []
img_names = []

import_images(img_dir, imgs, img_names)
augment_images(imgs, img_names)